In [ ]:
# ========== ----- ========== Import Libraries ========== ----- ========== #


import sklearn
print(sklearn.__version__)

import numpy as np
# from matplotlib import pyplot as plt
import dlib
import cv2
import os
import re
import json
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from joblib import dump, load

# ========== ----- ========== End ========== ----- ========== #

In [ ]:
# ========== ----- ========== Local Derivative Pattern Services For Image ========== ----- ========== #

def F(a, b):
    if a*b > 0:
        return "0"
    elif a*b <= 0:
        return "1"


In [ ]:
def I(picture, h, w, theta):
    if (theta == 0):
        return picture[h,w] - picture[h,w+1]
    elif (theta == 45):
        return picture[h, w] - picture[h-1, w+1]
    elif (theta == 90):
        return picture[h, w] - picture[h-1, w]
    elif (theta == 135):
        return picture[h, w] - picture[h-1, w-1]


In [ ]:
def ldp_pixel(picture, h, w):  # calculating local derivative pattern value of a pixel
    eigth_bit_binary = []
    four_decimal_val = []
    decimal_val = 0
    angles = [0, 45, 90, 135]

    for theta in angles:

        # starting from top left,assigning bit to pixels clockwise at theta degree
        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h-1, w-1, theta)))

        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h-1, w, theta)))

        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h-1, w+1, theta)))

        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h, w+1, theta)))

        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h+1, w+1, theta)))

        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h+1, w, theta)))

        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h+1, w-1, theta)))

        eigth_bit_binary.append(
            F(I(picture, h, w, theta), I(picture, h, w-1, theta)))

        l = "".join(eigth_bit_binary)
        decimal_val = int(l, 2)
        four_decimal_val.append(decimal_val)
        eigth_bit_binary = []

    return four_decimal_val


In [ ]:
def ldp_img(img):
    m, n = img.shape
    # converting image to grayscale
    # ldp_photo = np.zeros((m, n),np.uint8)
    ldp_img = np.zeros((m, n, 4))
    # converting image to ldp
    for i in range(2, m-2):
        for j in range(2, n-2):
            ldp_pixels = ldp_pixel(img, i, j)
            ldp_img[i, j, 0] = ldp_pixels[0]
            ldp_img[i, j, 1] = ldp_pixels[1]
            ldp_img[i, j, 2] = ldp_pixels[2]
            ldp_img[i, j, 3] = ldp_pixels[3]

    return ldp_img

# ========== ----- ========== End ========== ----- ========== #

In [ ]:
# ========== ----- ========== Local Derivative Pattern Main Function For Image ========== ----- ========== #

def extract_hist(gray_scale_img):

    features = ldp_img(gray_scale_img)

    histogram_0, bin_edges = np.histogram(
        features[:, :, 0], bins=256, range=(0, 256))

    histogram_45, bin_edges = np.histogram(
        features[:, :, 1], bins=256, range=(0, 256))

    histogram_90, bin_edges = np.histogram(
        features[:, :, 2], bins=256, range=(0, 256))

    histogram_135, bin_edges = np.histogram(
        features[:, :, 3], bins=256, range=(0, 256))

    hist = np.concatenate((histogram_0, histogram_45, histogram_90, histogram_135))

    return hist

# ========== ----- ========== End ========== ----- ========== #

In [ ]:
# ========== ----- ========== Local Derivative Pattern Main Function For TOP ========== ----- ========== #

def LDP_TOP(frames):

    x, y, z = frames.shape
    plane_yz = frames[(x//2), :, :]
    plane_xz = frames[:, (y//2), :]
    plane_xy = frames[:, :, (z//2)]

    hist_yz = extract_hist(plane_yz)
    hist_xz = extract_hist(plane_xz)
    hist_xy = extract_hist(plane_xy)
    
    hist_TOP = np.concatenate((hist_yz, hist_xz, hist_xy))
    return hist_TOP

# ========== ----- ========== End ========== ----- ========== #


In [ ]:
# ========== ----- ========== Reading Training Data ========== ----- ========== #

# Reading Training Data
train_frame_folders = [
    'faceforensics_dataset_train/original_sequences',
    'faceforensics_dataset_train/manipulated_sequences/Deepfakes',
    # 'faceforensics_dataset_train/manipulated_sequences/Face2Face',
    # 'faceforensics_dataset_train/manipulated_sequences/FaceSwap',
    # 'faceforensics_dataset_train/manipulated_sequences/NeuralTextures'
]


In [12]:

# Face detector
detector = dlib.get_frontal_face_detector()

list_of_train_LDP = []
list_of_train_labels = []

for train_frame_folder in train_frame_folders:
    list_of_train_data = [f for f in os.listdir(
        train_frame_folder) if f.endswith('.mp4')]

    for vid in list_of_train_data:
        frames = []
        # Video capturing constructor.
        cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))

        # (CAP_PROP_FPS) Returns frame rate of the video (#frames / second).
        frameRate = cap.get(5)
        while cap.isOpened():  # Returns true if video capturing has been initialized already.

            # (CAP_PROP_POS_MSEC) Current position of the video file in milliseconds or video capture timestamp.
            frameId = cap.get(1)
            ret, frame = cap.read()  # The methods/functions combine VideoCapture::grab and VideoCapture::retrieve in one call. This is the most convenient method for reading video files or capturing data from decode and return the just grabbed frame. If no frames has been grabbed (camera has been disconnected, or there are no more frames in video file), the methods return false and the functions return NULL pointer.
            if ret != True:
                break

            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()

                crop_img = frame[y1:y2, x1:x2]
                if crop_img is not None and crop_img.size > 0:
                    crop_img = cv2.resize(crop_img, (128, 128))
                    gray_scale_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                    frames.append(gray_scale_img)
            
            # d = 3 seconds.
            if frameId % ((int(frameRate)+1)*3) == 0:
                if (len(frames) > 0):
                    frames_ldp = LDP_TOP(np.array(frames).astype(np.float64))
                    list_of_train_LDP.append(frames_ldp)
                    if train_frame_folder == 'faceforensics_dataset_train/original_sequences':
                        list_of_train_labels.append(0)
                    else:
                        list_of_train_labels.append(1)
                frames = []


# ========== ----- ========== End ========== ----- ========== #


In [26]:
# ========== ----- ========== SVM Model Training ========== ----- ========== #

# param_grid = {'C': [0.1, 1, 5], 'gamma': [1, 0.1, 0.01],
#               'kernel': ['linear', 'rbf']}

model = svm.SVC(kernel='rbf')
# grid = GridSearchCV(model, param_grid, refit=True, n_jobs=-1, verbose=1)
model.fit(list_of_train_LDP, list_of_train_labels)

# Save the trained model to a file
dump(model, 'Deepfakes.joblib')

# Load the saved model from the file
# loaded_model = load('svm_model.joblib')

# ========== ----- ========== End ========== ----- ========== #


['Deepfakes.joblib']

In [27]:
# ========== ----- ========== SVM Model Training ========== ----- ========== #

# param_grid = {'C': [0.1, 1, 5], 'gamma': [1, 0.1, 0.01],
#               'kernel': ['linear', 'rbf']}

model = svm.SVC(kernel='linear')
# grid = GridSearchCV(model, param_grid, refit=True, n_jobs=-1, verbose=1)
model.fit(list_of_train_LDP, list_of_train_labels)

# Save the trained model to a file
dump(model, 'Linear-Deepfakes.joblib')

# Load the saved model from the file
# loaded_model = load('svm_model.joblib')

# ========== ----- ========== End ========== ----- ========== #
